<a href="https://colab.research.google.com/github/lookhmoo/Thesis-Data/blob/main/predata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
from pandas.core.indexes.datetimes import date
import datetime as dt
import copy

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)


def convert_time(datestring="10.00"):
    d = datetime.strptime(datestring, "%H:%M")
    return d.strftime("%-I:%M %p")
    
    
# result = get_temp(start_date, end_date, select_hours=[7,8,9,10])
# start_date = date(2022, 1, 1)
# end_date = date(2022, 1, 2)
# select_hours="07:00"


def get_temp(start_date, end_date, select_hours=[]):
    
    edf = pd.DataFrame(columns = ['Time', 'Temperature', 'Dew Point', 'Humidity', 'Wind', 'Wind Speed', 'Wind Gust', 'Pressure', 'Precip.', 'Condition', 'dates']) 
    for single_date in daterange(start_date, end_date):
        date = single_date.strftime("%Y-%m-%d")
        url = 'https://www.wunderground.com/history/daily/th/mueang-phitsanulok/VTPP/date/'+date
        driver.get(url)
        tables = WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))
        for table in tables:
            newTable = pd.read_html(table.get_attribute('outerHTML'))      
            
            if newTable:
                data = newTable[0]
                data['dates'] = pd.Timestamp(date)
                data_tmp = data.loc[data['Time'] == str(convert_time(select_hours))]
                edf = pd.concat([edf, data_tmp])
    return edf.reset_index()

df = pd.read_csv('https://raw.githubusercontent.com/lookhmoo/Thesis-Data/main/ElectricityProfileSGtech_22042022_1.csv', parse_dates = [ 'new_datetime'])  
df = df[df.name == 'NDIR-SERT']
df = df.reset_index()

df_tmp  = copy.deepcopy(df)
df_output = pd.DataFrame()

for i, row in df.iterrows():
    hour = str(row['hour']) + ":00"
    date = row['new_date']
    start_date = dt.datetime.strptime(date, "%Y-%m-%d").date()
    end_date = start_date + dt.timedelta(days=1)
    temp = get_temp(start_date, end_date, hour)
    result = pd.concat([df_tmp.iloc[[i]].reset_index(), temp], axis=1, ignore_index=False)
    result = result.drop(['index', 'dates', 'new_datetime', 'Time'], axis=1)
    df_output = pd.concat([df_output, result], ignore_index=True)  

tempdf = df_output['Temperature'].str.split(r"\s+", expand=True)
tempdf.columns=['temperature(°F)', 'd']
tempdf = tempdf.drop(['d'], axis = 1)

dewdf = df_output['Dew Point'].str.split(r"\s+", expand=True)
dewdf.columns=['dew_point(°F)', 'd']
dewdf = dewdf.drop(['d'], axis = 1)

hudf = df_output['Humidity'].str.split(r"\s+", expand=True)
hudf.columns=['humidity(%)', 'd']
hudf = hudf.drop(['d'], axis = 1)

wsdf = df_output['Wind Speed'].str.split(r"\s+", expand=True)
wsdf.columns=['wind_speed(°mph)', 'd']
wsdf = wsdf.drop(['d'], axis = 1)

wgdf = df_output['Wind Gust'].str.split(r"\s+", expand=True)
wgdf.columns=['wind_gust(°mph)', 'd']
wgdf = wgdf.drop(['d'], axis = 1)

predf = df_output['Pressure'].str.split(r"\s+", expand=True)
predf.columns=['pressure(°in)', 'd']
predf = predf.drop(['d'], axis = 1)

pcdf = df_output['Precip.'].str.split(r"\s+", expand=True)
pcdf.columns=['precip(°in)', 'd']
pcdf = pcdf.drop(['d'], axis = 1)

dfall = pd.concat([tempdf, dewdf, hudf, wsdf, wgdf, predf, pcdf], axis=1)

df2k = pd.concat([df_output, dfall], axis=1)
df2k = df2k.drop(['Temperature', 'Dew Point', 'Humidity', 'Wind Speed', 'Wind Gust', 'Pressure', 'Precip.'], axis=1)

df2k['temperature(°F)'] = df2k['temperature(°F)'].astype('float')
df2k['dew_point(°F)'] = df2k['dew_point(°F)'].astype('float')
df2k['humidity(%)'] = df2k['humidity(%)'].astype('float')
df2k['wind_speed(°mph)'] = df2k['wind_speed(°mph)'].astype('float')
df2k['wind_gust(°mph)'] = df2k['wind_gust(°mph)'].astype('float')
df2k['pressure(°in)'] = df2k['pressure(°in)'].astype('float')
df2k['precip(°in)'] = df2k['precip(°in)'].astype('float')

df2k['temperature(°C)'] = ((df2k['temperature(°F)'] - 32) * 5 / 9).round(2)
df2k['dew_point(°C)'] = ((df2k['dew_point(°F)'] - 32) * 5 / 9).round(2)

df2k.dropna(inplace=True)